In [1]:
import pandas as pd
import numpy as np
import time

In [2]:
df = pd.read_csv("./data/transform_sample_data.csv")
df.head()

,password,strength,len,alphaUC,alphaLC,number,symbol,midChar,repChar,uniqueChar,consecAlphaUC,consecAlphaLC,consecNumber,consecSymbol,seqAlpha,seqNumber,seqKeyboard
0,csillik,0.180594,7,0,7,0,0,0,2,5,0,1,0,0,0,0,0
1,huniihuu,0.177778,8,0,8,0,0,0,4,4,0,2,0,0,0,0,0
2,chaipy,0.172331,6,0,6,0,0,0,0,6,0,0,0,0,0,0,0
3,876876b,0.155556,7,0,1,6,0,5,3,4,0,0,0,0,0,1,0
4,miiwhy,0.154795,6,0,6,0,0,0,1,5,0,1,0,0,0,0,0
